In [97]:
from sage.all import *
import struct

DR = RealField(53)

DD = RealField(190)

def double_to_hex(f):
    packed = struct.pack('>d', float(f))
    return '0x' + packed.hex()

def split_double_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_lo = x - DD(x_hi)
    return (x_lo,x_hi)

def split_triple_double(x):
    # Split RR value x into hi + lo (double-double)
    x = RealField(190)(x).exact_rational()
    x_hi = DR(x)  # convert to f64
    x_mid = DR(x - DD(x_hi))
    x_lo = x - DD(x_hi) - DD(x_mid)
    return (x_lo, x_mid, x_hi)

def print_double_double(mark, x):
    splat = split_double_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}),")

def print_triple_double(mark, x):
    splat = split_triple_double(x)
    print(f"{mark}({double_to_hex(splat[0])}, {double_to_hex(splat[1])}, {double_to_hex(splat[2])}),")

def format_dyadic_hex(value):
    l = hex(value)[2:]
    n = 8
    x = [l[i:i + n] for i in range(0, len(l), n)]
    return "0x" + "_".join(x) + "_u128"

def print_dyadic(value):
    (s, m, e) = RealField(128)(value).sign_mantissa_exponent();
    print("DyadicFloat128 {")
    print(f"    sign: DyadicSign::{'Pos' if s >= 0 else 'Neg'},")
    print(f"    exponent: {e},")
    print(f"    mantissa: {format_dyadic_hex(m)},")
    print("},")

In [116]:
from sage.all import *
from mpmath import mp, besseli, taylor

# High precision
mp.prec = 500

# Step 1: Define I0(x) - 1
def shifted_i0(x):
    return besseli(0, x) - mp.mpf(1)

# Step 2: Series of I0(x) - 1 in terms of (x/2)^2
terms = 90
from mpmath import taylor
coeffs = taylor(shifted_i0, 0, terms)

# print(coeffs)

# Step 3: Build series in terms of y = (x/2)^2
R = PolynomialRing(RealField(500), 'y')
y = R.gen()
f = R(0)

for n in range(2, terms, 2):
    k = n // 2
    c = RealField(500)(coeffs[n])
    if n >= 1:
        f += R(c) * y**(k-1) * (4**k)
    else:
        f += R(c) * y**(k-1) * (4**k)

print(f)

x = RealField(107)(3.5593750000924014)

print(f(x**2/4)*(x**2/4) + 1)

# rat_approx = SR(f).power_series(RealField(107)).pade(28, 0)
# numer = rat_approx.numerator()
# denom = rat_approx.denominator()

# # print(numer)
# # print(denom)

# def I0_approx(y):
#     z = (y/2)**2
#     return 1 + z * numer(z) / denom(z)

# # Print result
# print("Rational approximation of I0(x):")
# # pretty_print(I0_approx)

# # Evaluate at some point
# print("I0_approx(1) =", I0_approx(-8.5))
# print("I0_real(1) =", besseli(0, -0.5))

# print("Numerator polynomial coefficients:")

# coeffs = numer.coefficients(sparse=False) 

# for i, c in enumerate(coeffs):
    # print("")
    # print_double_double("")
    # print("f64::from_bits(" + double_to_hex(c) + "),")

1.41514618630778449023136504789800043384960096405754551949596226075670317035088121461113881167744441986718863686989614761107450809403356362836400042839e-109*y^43 + 2.73972301669187077308792273273052883993282746641540812574418293682497733779930603148716473940753239686287720098011894177504024767004897918451270482937e-106*y^42 + 5.06574785786326905943956913281874782503579798540208962450099425018938309759091685221976760316452740179945994461223992334204941794192056251216399122950e-103*y^41 + 8.93597922127080662085139995029227116336314764624928609761975385733407178415037732731567005198222633677424734229599122477537517324954787227145728052886e-100*y^40 + 1.50213810709562259296512033164413078256134511933450499300988062341785746691567842872176413573821224721175097823995612488474056662324899732883196885690e-96*y^39 + 2.40342097135299614874419253063060925209815219093520798881580899746857194706508548595482261718113959553880156518392979981558490659719839572613115017104e-93*y^38 + 3.6556032974279071

In [4]:
from mpmath import mp, besseli

# Setup precision
mp.dps = 100

# Define the interval for y (example: from 0 to 3)
a = 0.0
b = 7.5

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(70)(max_rel_error).log2()}")

Maximum relative error on [0.0, 7.5] is about 105.43635783664410671


In [5]:
from mpmath import mp, besseli, nprint, taylor, chebyfit

# High precision
mp.prec = 1000

# Define the function to expand
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

def f_v(v):
    x = mp.mpf(1) / v
    return f(x)

# Generate coefficients of series in u = 1/x
terms = 80
coeffs = taylor(lambda u: f_v(mp.mpf(u)), mp.mpf('1e-70'), terms)

# print(coeffs)

def f_z(v):
    x = mp.mpf(1) / v
    return f(x)

# coeffs = chebyfit(lambda u: f_z(mp.mpf(u)), [1/9.5, 1/709], terms)
# coeffs = list(reversed(coeffs))
# print(cheby)

# print(coeffs)

# for n in range(terms):
#     u = mp.mpf('1e-30')  # Very small u ≈ 1/x
#     deriv = mp.diff(lambda u: f(1/mp.mpf(u)), u, n)
#     coeff = deriv / mp.fac(n)
#     coeffs.append(coeff)
    # print(f"u^{n}: {coeff}")

x = var('x')
u = x
# shifted_poly = sum(RealField(120)(coeffs[k]) * (u - u0)**k for k in range(terms))
coeffs = [QQ(RealField(120)(coeffs[k]).exact_rational()) for k in range(terms)]
# print(shifted_poly)

# Construct polynomial approximation in 1/x
# P_ring = PolynomialRing(RR, 'u')
taylor_poly = sum(c * u**k for k, c in enumerate(coeffs))

# print(taylor_poly)

pade_approximant = SR(taylor_poly).power_series(RealField(120)).pade(26, 0)

expr_in_x = pade_approximant.subs(u = 1/x)

numer_asympt = expr_in_x.numerator()
denom_asympt = expr_in_x.denominator()

# print(numer_asympt)

# print(denom_asympt)

print(float(f(10.1)))
print(float(numer_asympt(RealField(53)(1/10.1))/denom_asympt(RealField(53)(1/10.1))))
print(taylor_poly(x=1/RealField(53)(1/10.1)))
print(expr_in_x(x=1/10.1))

coeffs = numer_asympt.coefficients(sparse=False) 

print("coeffs")

for i, c in enumerate(coeffs):
    # print_double_double("", c)
    print(RealField(53)(c))
    # print("f64::from_bits(" + double_to_hex(c) + "),")

0.404188505662345
0.40418850661261374
5.21818307638252e169
0.40418850661261374
coeffs
0.398942280401433
0.0498677850501791
0.0280506290907257
0.0292194053028393
0.0447422143699727
0.0906029840991947
0.228395022416720
0.689263549793316
2.42319216724213
9.72642411573575
43.8904888222576
219.951199666086
1212.02275649333
7283.79060392626
47409.6727701986
332262.789997809
2.49456672803042e6
1.99748762266554e7
1.69925162344811e8
1.53044438980294e9
1.45487869805642e10
1.45574469728145e11
1.52935905981443e12
1.68312613919795e13
1.93647168827514e14
2.32473426177431e15
2.90703548792066e16


In [7]:
from mpmath import mp, besseli

# Setup precision
mp.prec = 150

# Define the interval for y (example: from 0 to 3)
a = 9.0
b = 13.0

# Number of test points
N = 1000

# Use your RealField 300 for evaluation
RF = RealField(107)

# Prepare to store max relative error
max_rel_error = 0

def I0_approx_asympt(y):
    return numer_asympt(x=1/y) / denom_asympt(x=1/y) * RealField(53)(y).exp() / RealField(53)(y).sqrt()

for i in range(N+1):
    y_val = a + (b - a) * i / N
    y_rf = RF(y_val)
    
    approx_val = I0_approx_asympt(y_rf)
    true_val = mp.besseli(0, y_val)
    
    # Compute relative error
    rel_err = abs(approx_val - true_val) / abs(true_val)
    
    if rel_err > max_rel_error:
        max_rel_error = rel_err

print(f"Maximum relative error on [{a}, {b}] is about {-RealField(107)(max_rel_error).log2()}")

Maximum relative error on [9.0, 13.0] is about 24.44990328711527391079110708667


In [6]:
from mpmath import mp, besseli, nprint

# High precision
mp.prec = 150
print(besseli(0, 16.225000000016742))

1110969.1973027737067063330973042313994430942


In [7]:
from mpmath import mp, besseli, chebyfit
from sage.all import *

# Set high precision in mpmath
mp.prec = 53

# Step 1: Define the function
def f(x):
    return mp.sqrt(x) * mp.exp(-x) * besseli(0, x)

coeffs = chebyfit(lambda u: f(mp.mpf(u)), [8, 709.782712893384], 27)
print(coeffs)

for i, c in enumerate(reversed(coeffs)):
    # print_double_double("", c)
    # print(c)
    print("f64::from_bits(" + double_to_hex(c) + "),")


[mpf('5.1033286636335007e-65'), mpf('-4.9448046485473206e-61'), mpf('2.2644503491559604e-57'), mpf('-6.5181879689808576e-54'), mpf('1.323100797534861e-50'), mpf('-2.0141049641424839e-47'), mpf('2.387233204663285e-44'), mpf('-2.2578753029007174e-41'), mpf('1.7327243128674407e-38'), mpf('-1.0912375418023538e-35'), mpf('5.6826614926496948e-33'), mpf('-2.4583026152180637e-30'), mpf('8.8538394385740462e-28'), mpf('-2.6551324549089779e-25'), mpf('6.6174596019044181e-23'), mpf('-1.3655130349836739e-20'), mpf('2.3193546998441735e-18'), mpf('-3.2166452699116028e-16'), mpf('3.6040093138654318e-14'), mpf('-3.217796031208781e-12'), mpf('2.249536548765481e-10'), mpf('-1.2039009184205734e-8'), mpf('4.789637519369099e-7'), mpf('-1.3621786829552683e-5'), mpf('0.00026227316601194486'), mpf('-0.0031505266377368203'), mpf('0.4194041232607285')]
f64::from_bits(0x3fdad784644d97aa),
f64::from_bits(0xbf69cf221bf9e8bb),
f64::from_bits(0x3f313036abaf0acd),
f64::from_bits(0xbeec912420536339),
f64::from_bits(0x3

In [129]:

values = [
' 0.398942280401432677964341644122557802627333118498687252492603505883893520413138122564281324036912444797339698671555264186775103361360715820992096349983 ',
' -0.14960335515053732675306211228440521403472993299464800400386751537765011687127248727751285965644485934818368878472985613656949680104658507946647085998115 ',
' -4.675104848444822027747244882312318254231208303502062688017923813003702284491406389939413585929783825070110901152198363858621714582521484239047800495984e-2 ',
' -4.090716749642786858428805139676753258398033342820791069702455015066194114890713292185044808091916579226813147702353050201063950461366442686220247049164e-2 ',
' -5.752566750110616835491030603057571044993251443734532498913771087968773809207374610540246089631937030292879610124050263754846266221223313807512968904302e-2 ',
' -0.11075014930080667053830773405150947677413968101157541100566091054736292483262865613347493262739844375310638207193079152335412650330322947642641743608393 ',
' -0.2664148534919718903603714342166176406607192417301439831419841909450470321219136192149909666173157295939904065607286252857003317937441418565617982274072 ',
' -1.511185511226273857945351204146405670460754119351473591389799960818330457990497330038306022860557953703510279876451085856685172101235831498467693910967 ',
' 112.15649203912961691175142657691100760757949205263021319427453242169428184849761429267944336827505248856670441399636076943399818049769746605264168166867 ',
' -14799.472162610031963295618856395478130318051852344028010806901391805835431473772271882179637929587351625818237678588422966544976226429398086526137447144 ',
' 1.550637427677587386318739720705130150267491455106536866259744009946148580197366299542353000877351891361846348011988271535585110052355584243516936855506e6 ',
' -1.341904963993670626464606621517162343939602947751588762491734178522376989057989590707430225105685651575447913135293082355799648204987559944617147613144e8 ',
' 9.68469163789066371822376158026870152779171078370059395476539005982131986812050465248850328239384943740481284620379102590669532921878832184262453727692e9 ',
' -5.87933568631927881997178462530423200807906211111251792719380474709887018968286865707487082384275304017368238010719789424112897082707987635635140205975e11 ',
' 3.022850326454256261731899965478439310980799984153618118973304358564970602271355510631825297126427618066946436758704097868022751059021363122766280559536e13 ',
' -1.3232675484055049154912341503415925917503429975939277518672406031503786427310861159546272178149934356381233105476217192008770804489029511917667717706816e15 ',
' 4.950752529288293650455242664753844571008635428325867580498853233994363004882945996461723626288739934416343043216076891405429483582541640191344897466335e16 ',
' -1.58658700110977507861555998249584668987411303398716754047295692027473310974035495549326941921596365158438105235285837554505709443763824099915789803723e18 ',
' 4.357400916276874066775738473515916653894028682817555442842233781175457885442252828582577280996750929125434252435420277461467000804551283780267201146172e19 ',
' -1.0236664758872984368772945166487575962795016055073141248022602466410231036240017282822943923633258900625816863323364374207649396886347424892833951714463e21 ',
' 2.046242178336280531985763033813351436027359767298102274949600133183981429877910144429839608673540759714634059924895303085094766914109516786511981917724e22 ',
' -3.441371291293628585039928800641466209687336090279628048174256619450968973812134081691035984672418336469462516516137867812831635111444944627299070604416e23 ',
' 4.755777173213815078606461364569340598269320799954242942578553687409069231930313194010095281880651649028970086153273283580969879319307930727143054374e24 ',
' -5.10479770400997943407123257742590146246941032042053693183050810314349477752688712146261594073112824395595904531290274131142450093299660807260069468779e25 ',
' 3.5212200417659295432824464526060041005855273947990541263771518061014284395127630554192181291681822562563965761512667317388039262772633019436545788908e26 ',
' 3.519291870899108677290234465564863456693079899210561321638641401468444933273012471680527363833435249522662109662385352611316219036588026981203253744399e26 ',
' -5.69921032614458889685521903287314011357891169707015939312604271342007269490728292581652120498435650246158217037585938351787940005180110506623661118288e28 ',
' 1.06945736818499923488313645281310924196689229841144790055810271626173690293548238505819682365119253124068545378375589019742710842583598909071623276278e30 ',
' -1.347107455169668162467925591109513952465736396757502904433545208841589111264730289410299124000103114322670415017670603947955358502578186785690029648604e31 ',
' 1.322511257017707558037259449749421226282017646594135784223788883454181706284437923388095145897672034724735169216319618216893126860460334842568070171799e32 ',
' -1.0585964841467849218271295933140863749480102950571639576723287523929366874838917100358688670535080353946507344416403817146877515597972729549094462551413e33 ',
' 7.022968351586757939091777563386182240312660608944679597101489664102159327274258130940947406911644384960756507294240606965468538142736370534809637341287e33 ',
' -3.877629121179712117321664983961406249389497705027001474334292770191821397746361017534405904752727973862011838778508107126723821243460848128447497529292e34 ',
' 1.776137380424610896516608695060952977811618941805289398045348773409638831992987927102809380472628738680224129435055572334983852758800949172229376142719e35 ',
' -6.687323719265151351004629800455892162780502121601761829805322091235639129005254880469938831092953768048652949251215827780204438985122823507433088143627e35 ',
' 2.0374457890827161165883576491881813355294588762607790828386174609313264597877237623867179902989159370702834808012072667853201173482597845656748405344226e36 ',
' -4.902819619325272071518003923677018855229668367705800504155030763727507766635653191105448889267300377569059334438487554266908240121889966249946765196953e36 ',
' 8.974347935268347177487753450550595055264678529179622586878139122118696436221710122143044044761106851376100710757319682044666306653261697155559402285575e36 ',
' -1.1746575850834917204682653232915960667138230411517841312095018778476679363560927935721758922038585885618159941251696993519618320538058909025786908360263e37 ',
' 9.79580078316008992068779314219002070293074534229889388845740812223981103009179739861566177680733294549498961410881177415295407035686013365991177451554e36 ',
' -3.911532391131521288336512543207965165733820155503763067370021074522165023388161128301562162200101245235100173300547735141652302385953863993859420018725e36 ',
]


for i, c in enumerate(values):
    c = RealField(1000)(c.replace(" ", ""))
    # print_triple_double("", c)
    # print(RealField(53)(c))
    # print("f64::from_bits(" + double_to_hex(c) + "),")
    # print(double_to_hex(c) + ",")
    print_dyadic(c)

DyadicFloat128 {
    sign: DyadicSign::Pos,
    exponent: -129,
    mantissa: 0xcc42299e_a1b28469_64c2c9b9_bf441355_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -130,
    mantissa: 0x99319f36_f945f835_d047b4d0_772fc356_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -132,
    mantissa: 0xbf7e0704_b5ecff21_4a89e098_758e73fb_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -132,
    mantissa: 0xa78e4629_1b3f03c4_84592616_f87ad1d0_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -132,
    mantissa: 0xeba008c9_90accaa6_1ae9937a_152a6453_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -131,
    mantissa: 0xe2d0f96a_3151dc1a_3d7b6ac9_30a32982_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -129,
    mantissa: 0x88678715_d5533c7e_d19f6e4d_d2977f5c_u128,
},
DyadicFloat128 {
    sign: DyadicSign::Neg,
    exponent: -127,
    mantissa: 0xc16e86de_73f41b92_3f505d66_bccc2a21_u128,
},


In [9]:
# Use arbitrary precision and special functions

# Set the target overflow limit for f64
f64_max = RealField(400)(2)**1024 * (RealField(400)(1) - RealField(400)(2**-53))  # == f64::MAX

# Use high precision for computations
R = RealField(400)

# Define the function to compare I0(x) with f64::MAX
def bessel_diff(x):
    return besseli(1, R(x)) - f64_max

# Use binary search to find the cutoff value where I0(x) ≈ f64::MAX
def find_cutoff(x_lo=700, x_hi=720, tol=1e-12):
    while x_hi - x_lo > tol:
        x_mid = (x_lo + x_hi) / 2
        if bessel_diff(x_mid) > 0:
            x_hi = x_mid
        else:
            x_lo = x_mid
    return (x_lo + x_hi) / 2

cutoff = find_cutoff()
print("Cutoff x where I0(x) ≈ f64::MAX is:", cutoff)
print("I0(cutoff) =", bessel_I(1, cutoff))


Cutoff x where I0(x) ≈ f64::MAX is: 713.9876098185423
I0(cutoff) = inf


In [10]:
from sage.all import *
from mpmath import mp, taylor

mp.prec = 500

print(float(mp.besseli(1, -0.00019641107078895875)))

def f(x):
    return mp.besselj(1, x) 

terms = 30
coeffs = taylor(f, mp.mpf(0), terms)

prec_bits = 350
RR = RealField(prec_bits)
L = LaurentSeriesRing(RealField(500), 'x', default_prec = 500)
x = L.gen()

series = L(0)
for n, c in enumerate(coeffs):
    if c != 0:
        series += RR(c) * x**(n)

# Optional: print the series
print(series(subs=x).truncate(20))


-9.820553586804254e-05
0.500000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^2 - 0.0625000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^3 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^4 + 0.00260416666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666666667*x^5 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^6 - 0.0000542534722222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222222*x^7 + 0.000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000*x^8 + 6.7816840277777777777777777777777777777777777777777777777777

In [11]:
from mpmath import mp, taylor

import numpy as np

print(mp.besselk(1, 0.00025238024))
print(np.float32((mp.besselk(1, 0.00025238024))))

3962.27421187039566410098264693680058081741191737161137607427821099428075085462880283537778787967226526477089756964874964578129485634023301450824426594
3962.2742


In [96]:
R = PolynomialRing(RealField(107), 'z')
z = R.gen()
x = 1.2533141373155002419035727829162900815285534392705444 + 10.635433183254121541220236246429533099101008352895388*z + 20.174562190882996054154135155495106754234956241084497 *z**2 - 2.0124074566655218437700226406848876215293993602012009 *z**3 + 0.85730507521885754875680381278184681499877612573590721* z**4 - 0.64425895157008493270742725518393343576294899792536059* z**5 + 0.67053819037846809445676366792093172873542726698912348* z**6 - 0.85413500472728190956077165616865651403453673819423044* z**7 + 1.2239367955534735097474473321330242560821280353852232 *z**8 -  1.8447712859556259875259980609771210851930600191167760 *z**9 +  2.7636535917052952197873013830688588947414821008593453 *z**10 - 3.9240434118934010665799546313458516292033791432973163 *z**11 + 5.0777172436286910307426635134078788938504524879473308 *z**12 - 5.7974913539397550997755262645955779659647166218306631 *z**13 + 5.6807429630004755423328685794483075919280929751669777 *z**14 - 4.6555189128490244724632393339598798027214431882326775 *z**15 +  3.1064686409479559680372158221454582538304036291933162 *z**16 -  1.6349690402036198588459906106055152603676933476862251 *z**17 + 0.65042065733394472693321209960702682292340646867511452* z**18 - 0.18325191162927484578925149742335792209585086825186654* z**19 + 0.032510724379786101549632646211334498315782854292177549 *z**20 - 0.0027260051898979127045433464129385581691757697459122891* z**21

P = x.polynomial(z)

# Extract coefficients as a list (ordered from constant term up)
coeffs = [
' 1 ',
' 0.2499999999999999999999999999999999954006398482124599194658581362981297299288059340636026342068542918429624073175208953667691957889473997056484222412109 ',
' 2.77777777777777777777777777777779691931057089555688254476903583087425550704658010313989585080794886114469201066855795229787462474746462248731404542923e-2 ',
' 1.7361111111111111111111111111083004641876622300207416350126643766531079051266583722792156723858008551905259245547125945287846027209610610952950082719326e-3 ',
' 6.944444444444444444444444446547422857848814789533666278416276638600579676108543206234084609030897084151816846771933772035096527064368387982540298253298e-5 ',
' 1.929012345679012345679012251318302684844315147660816084767873549626042659799595236076632643859027195702321306916064397740599831339303182886624199454673e-6 ',
' 3.936759889140841521793930588071439996716742969653087403746571063490704653742633427847492734103240103984133465321259720264719616752199099485309830015467e-8 ',
' 6.151187326782564877797247288072149460333301837929849320461467733327149067754538827489348188714675304236248535340218559583344139042997166834381150835043e-10 ',
' 7.59405842812662330678280056306543904207663380898714703120703311355224377389199667855776961967486296162483555218561086724415820180284647251262031275587e-12 ',
' 7.594058428126623211353407535366944834429861161552691679729553267607394132971109156229687563756899489900895310662989882554737768322720364007899908169452e-14 ',
' 6.276081345559200750157160630118735839703575667375686018885736408939801200204179221535837289293413464257922384851161778471955454750273136161577529210575e-16 ',
' 4.358389823304485809667451203457575737516247109083343742934392026855861902111321423544327710398988402387019969213041766601799169121089745403912308824028e-18 ',
' 2.578928889554674836368448293552367560015702847774685000965167273091671554531201150949297716969649361769703643736780385024309605445169106252909680183287e-20 ',
' 1.315780044730846415792433764796279506444104384198536569721627883269191626591851968368027879685455532662490895689435640408963602771219272011653015815669e-22 ',
' 5.847911585784156401493223835488973461978664993227708417243939682989885197587857635925650830536313342106109582471288674853032569703682985678440464272073e-25 ',
' 2.284334552756511974922074599668368568545891803472044978289392187534382490807474984740544159804559096592445253613911158104575426115044909120983472133658e-27 ',
' 7.905186142402312169455177877247005089487039105043940684023285430414697711456317759994594454458187857317972741625982855491605427333327287260649597960184e-30 ',
' 2.43022691097561310570199542714883058212937226328929456159377849072413316726233205435608517759663921127459909548186880852041951238010190028326350362274e-32 ',
' 7.352659820595603251423985548963728616245901182827075057661050870160875064227941943171851185349618466945141639443064810518998219842943800673899015612516e-35 ',
]

# Print nicely
for c in coeffs:
    # print("f64::from_bits(" + double_to_hex(c) + "),")
    # print_double_double("", c)
    print_triple_double("", c)
    # print(c)

print("--")
print("f64::from_bits(" + double_to_hex(RealField(107)('1')) + "),")
print("f64::from_bits(" + double_to_hex(RealField(107)('8.6108479343670076529219106105176467832092478822742928')) + "),")
print("f64::from_bits(" + double_to_hex(RealField(107)('17.103015184855466847985374789136626320766401206703402')) + "),")

TypeError: unsupported operand type(s) for -: 'str' and 'sage.rings.real_mpfr.RealNumber'

In [84]:
# K1 series

from mpmath import mp, euler

V = PowerSeriesRing(RealField(150), 'z')

z = V.gen()

mp.prec = 150

R = RealField(160)

euler_gamma = R(euler)

lg = R(2).log()
lg4 = R(4).log()
lg8 = R(8).log()

expr = (z**17 * (R(-6989) + R(2520) * euler_gamma - R(2520) * lg))/R('4832746593583104000') \
        + (z**15 * (R(-1487)+R(560) * euler_gamma-R(560) *lg))/R('3728971137024000')\
        +(z**13 * (R(-353)+R(140) * euler_gamma-R(140) * lg))/R('4161798144000')\
        +(z**9 * (R(-131)+R(60) * euler_gamma-R(60) * lg))/R('88473600')\
        +(z**11 * (R(-71)+R(30) * euler_gamma - R(30) *lg))/R('5308416000')+(z**7 * (R(-47)+R(24) * euler_gamma - R(24) * lg))/R(442368)\
        +R(1/R(64)) * z**3 * (R(-5)+R(4) * euler_gamma-R(4) * lg)+ R(1/4) * z * (R(-1)+R(2) * euler_gamma-lg4)+(z**5 * (R(-5)+R(3) *euler_gamma-lg8))/R(1152)\
        + (z**21 * (R(-82451)+R(27720) * euler_gamma-R(27720) * lg))/R('8420577664659200409600000')\
        + (z**23 * (R(-42433)+R('13860') * euler_gamma-R('13860') * lg))/R('2223032503470028908134400000')\
        + (z**25 * (R('-1132133')+R('360360') * euler_gamma-R('360360')* lg))/R('36066479336297749005572505600000')\
        + (z**27 * (R('-1158863')+R('360360') * euler_gamma - R('360360') * lg))/R('26256396956824761276056784076800000')
print(expr)

coeffs = expr.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr2 = -euler_gamma + lg + 1/4 *  z**2 *(1 - euler_gamma + lg) + (\
 z**12 *(49 - R(20) * euler_gamma + R(20) *lg))/42467328000 + (\
 z**10 *(137 - 60 *euler_gamma + 60 *lg))/884736000 + (\
 z**14 *(363 - 140* euler_gamma + R(140) *lg))/R(58265174016000) + (\
 z**16 *(761 - 280 *euler_gamma + 280 *lg))/R(29831769096192000) + (\
 z**18 *(7129 - 2520 *euler_gamma + 2520 *lg))/R(86989438684495872000) +\
 1/128* z**4 *(3 - 2 * euler_gamma + lg4) + (\
 z**6 *(11 - 6 * euler_gamma + R(64).log()))/R(13824) + (\
 z**8 *(25 - R(12) * euler_gamma + R(4096).log()))/R(1769472)

print("s")

coeffs = expr2.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

R = RealField(200)

expr3 = R('1.2533141373155002546825431555831963990977483071423532')+R('11.581464036148641246581323440149428509260059307098616') * z+\
R('26.789405266295715810558337396548064861031194327368438')* z**2 + R('7.3649417349531436967188010018370139100180573487266502') * z**3 \
- R('1.7096604692508150493340167930493663713673704318166940') * z**4+R('1.0804255276094591411415638947116978390123141265771834') *z**5\
+ R('-1.0469297880831626083930679006829252269737874877936028') * z**6+R('1.3002369515375729922365615519495020332408157597613544') * z**7 \
+ R('-1.8716868006869984282123524738168938690121343397007868') * z**8+R('2.9031694730522518237131899238517881330616803775751557') * z**9\
+R('-4.5732446918809058964898044936311043412823198783007045') * z**10+R('6.9647249644245100963438249211100614638042076172726253') * z**11\
+R('-9.8478668366535772967541646404708100660109176278818808') * z**12+R('12.509168035842933091158686315206698671310735894796194')*  z ** 13\
+ R('-13.889426789762708072171011520013585808954445304261549') * z**14+R('13.160468639020739474423102361556011097677163142977284') * z**15\
+ R('-10.397245062168499847721119740813368446763822759405222') * z**16+R('6.6791470271098701770545337257192390539281302865779416')* z**17\
+ R('-3.3836343614489495447896172378322198542839494659295979') * z**18+R('1.2963434524119789098189498423127080263478530079657214') * z**19\
+ R('-0.35208923494132138684480968071673794226763563154951347') * z**20+R('0.060290645206928280777309624029614864219397241979017862') * z**21
+ R('-0.0048861904873423508267690490483453010423241530137713923') * z**22

print("s")

coeffs = expr3.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

num_expr = R('1') + R('8.8656713459366445640627099502526076307634900011454201') * z + R('18.167413600340225608199066162434028713917004023936959') * z**2

print("s")

coeffs = num_expr.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr5 = R('1.2533141373155008574274621176544090441381452527447420') + R('9.6076379479355559677000707539871221691292104908849198') *z+ R('18.406096659958275387699898145740773014596806776002146')* z**2+ R('4.7300780306398066569023595697768560808449485831009526') * z**3 - R('1.0163754739336227088791404523051013401612222070368604') * z**4 + R('0.58130377839410367882508833965831895831753991773665285') * z**5 - R('0.49232098929036983678046222511723773395343872206074951') * z**6 + R('0.50845889394645196319680354438342957369507316973394789') * z**7 - R('0.57007995718786118274924078452898987222644530044713499') * z**8 + R('0.63665165434076501178161838064495936354739233673543394') * z**9 -R('0.66155559914107090752230117604161227614823201013482179')  *z**10 + R('0.60514236339380552468218358490833810798530137756542675') * z**11 - R('0.46461779005424436911181683303777110485398428882497891') * z**12 + R('0.28604062619504129802564994120724513947211384010839769') * z**13 - R('0.13409133022404639058502814678010866732272117796110097') * z**14 + R('0.044588941897052332965529317763488333331500636452569986') * z**15 - R('0.0093174632995096483384846461968063098276613230731306592') * z**16 + R('0.00091636018782473506248122477549734767637011116719735765') * z**17
print("s17")

coeffs = expr5.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr6 = R('1') + R('7.2907859844421536422466214803077115972643852182573845') * z + R('12.069083105317276679173254499363797483894674077685109') * z**2

print("s6")

coeffs = expr6.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr7 = R('1.2533141373155058832020565569172201040757811539960052') +\
R('8.7651973412137614264861540662854425414839427691011145') * z +\
R('15.301333383136817613847672052255163247878215452807969') * z**2 +\
R('3.7829834554855261433752777441348702648227134959720771') * z**3 -\
R('0.77593806509812267632818079807631581270345383814683018') * z**4 +\
R('0.41676154202785490700268735525325363401025859665280452') * z**5 -\
R('0.32321613121063810175040498613776183097624003643340030') * z**6 +\
R('0.29515078528633240747470434929670937103337640937811351') * z**7 -\
R('0.28002654532055288374464773714587315634778248247707827') * z**8 +\
R('0.25154585259411234638788530725428550333092636929147498') * z**9 -\
R('0.19864146379123211745095983722287988679278308045597601') * z**10 +\
R('0.12933371606491300620629445862636806968339959524282497') * z**11 -\
R('0.065132103886990260625436025198517761428939721122027237') * z**12 +\
R('0.023450256888942946510235248277178016489682020971444765') * z**13 -\
R('0.0053217258841870574666868147901043559815181047238250563') * z**14 +\
R('0.00056843502842415578402336351315730594994654329942957717') * z**15
print("s7")
coeffs = expr7.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr8 = R('1') + R('6.6186156309497610058422408472475779935941069007337244') * z + R('9.8439043035970074081829694480068799884816470174605034') * z**2

print("s8")
coeffs = expr8.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")


0.00000000000000000000000000000000000000000000 - 0.30796575782920622440536001568788706851667040*z + 0.00000000000000000000000000000000000000000000*z^2 - 0.085370719728650778050670001960985883564583800*z^3 + 0.00000000000000000000000000000000000000000000*z^4 - 0.0046421827664715601965556945261521895929687694*z^5 + 0.00000000000000000000000000000000000000000000*z^6 - 0.00011253607036630565224305845077631876466833084*z^7 + 0.00000000000000000000000000000000000000000000*z^8 - 1.5592887702038206530382306347039845583541356e-6*z^9 + 0.00000000000000000000000000000000000000000000*z^10 - 1.4030163700386777046923526894138142924556068e-8*z^11 + 0.00000000000000000000000000000000000000000000*z^12 - 8.8718962192938529705274625734613326595944513e-11*z^13 + 0.00000000000000000000000000000000000000000000*z^14 - 4.1617958191203952495707243361574556974960382e-13*z^15 + 0.00000000000000000000000000000000000000000000*z^16 - 1.5066271898317758599066165950845176173432059e-15*z^17 + 0.00000000000000000000000

In [159]:
expr_k0 = R('1.2533141373154948366080041951960582911146035121315740') +     \
R('8.2245773899079526320913715531861187489858334082181407') * z +  \
R('11.659225568069422689144269600379982654892649199544085') * z**2 -\
R('1.0798339813009586546858974989485617551600570757328948') * z**3 +\
R('0.41395488588116326367650677076555338420712511722896507') * z**4 - \
R('0.26882235446941534398867801042393895684894242532868239') * z**5 + \
R('0.22818681551596479442117213053367341432677188128277528') * z**6 - \
R('0.21895425336695130664551164207288095208476881669620451') * z**7 + \
R('0.21372536171853447391431857503665912965806458885073050') * z**8 - \
R('0.19513656746685727368530532818280649680550011563081530') * z**9 + \
R('0.15547852946970707334322463350358673750505169909153039') * z**10 -\
R('0.10167975086229470304199304100489815474211563951174081') * z**11 +\
R('0.051289281594646311701768857628948719930473359198687553') * z**12 -\
R('0.018464229923427865340456934685352521443259755705199029') * z**13 +\
R('0.0041851949014646505623635614938738768135156088980293181') * z**14 -\
R('0.00044620091699802778881987206405244817066758391926795877') * z**15

coeffs = expr_k0.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

expr_k0_den = R('1') + R('6.6872633185311663100009203016549852528062449248109539') * z + R('10.068311487338696570818625050453714367868312870255339') * z**2
    
print("expr_k0_den")
coeffs = expr_k0_den.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")

f64::from_bits(0x3ff40d931ff626ed),
f64::from_bits(0x402072fbcec226c2),
f64::from_bits(0x40275186037f1723),
f64::from_bits(0xbff146ffff2cc0e5),
f64::from_bits(0x3fda7e3ca23841ac),
f64::from_bits(0xbfd13462ad384f99),
f64::from_bits(0x3fcd3539bf027d79),
f64::from_bits(0xbfcc06b166c3fcd9),
f64::from_bits(0x3fcb5b5a47741483),
f64::from_bits(0xbfc8fa3c2bc30d81),
f64::from_bits(0x3fc3e6b86fa6ba6c),
f64::from_bits(0xbfba07af249e75b5),
f64::from_bits(0x3faa4296b62b2eb2),
f64::from_bits(0xbf92e8497eaaf439),
f64::from_bits(0x3f71247eb3aab51a),
f64::from_bits(0xbf3d3e0258904b11),
expr_k0_den
f64::from_bits(0x3ff0000000000000),
f64::from_bits(0x401abfc1f49353c1),
f64::from_bits(0x402422f9b9281f24),


In [91]:
Z = PowerSeriesRing(RealField(300), 'z', default_prec = 300)
R = RealField(170)
z = Z.gen()

from mpmath import *
mp.dps = 60;

euler_gamma = R(euler)

lg  = R(2).log()
lg4 = R(4).log()
lg64 = R(64).log()
lg4096 = R(4096).log()

expr2 = -euler_gamma + lg + R(1/4) *  z**2 *(1 - euler_gamma + lg) + (\
 z**12 *(R(49) - R(20) * euler_gamma + R(20) *lg))/R('42467328000') + (\
 z**10 *(R(137) - R(60) *euler_gamma + R(60) *lg))/R('884736000') + (\
 z**14 *(R(363) - R(140)* euler_gamma + R(140) *lg))/R('58265174016000') + (\
 z**16 *(R(761) - R(280) *euler_gamma + R(280) *lg))/R('29831769096192000') + (\
 z**18 *(R(7129) - R(2520) *euler_gamma + R(2520) *lg))/R('86989438684495872000') +\
 R(1/128)* z**4 *(R(3) - R(2) * euler_gamma + lg4) + (\
 z**6 *(R(11) - R(6) * euler_gamma + R(64).log()))/R(13824) + (\
 z**8 *(R(25) - R(12) * euler_gamma + R(4096).log()))/R(1769472)

coeffs = expr2.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")
        # print(f"z^{i}: {c}")1.25331413731550025097574185194058307743736824945827587299663 + 

expr3 = R('1.2533141373155002605829645596860784532244203418534036') +\
R('11.199806259196258030913593580690918511808102204545876') * z +\
R('25.047196019879293563662797795039110440533584729746710') * z**2 +\
R('6.8100094128136955990832848808138566898175532073827290') * z**3 -\
R('1.5614022027790387836235246367741667397668469085932065') * z**4 +\
R('0.97148914664460031939572797647132783696513822904722992')* z**5 -\
R('0.92226751784973237584058417530768480661385760935111574')* z**6 +\
R('1.1142224216363698749222831852221677739252013372252898')*z**7 - \
R('1.5453079477053328991519790838061388093855069739978008')*z**8 + \
R('2.2818288909050356344679754738410666675596287637645690')*z**9 - \
R('3.3753902052364075754201286422231714264549638618942536')*z**10 +\
R('4.7572556183821127508767403093723929650605681766385452')*z**11 -\
R('6.1320895049170075029455736013227131794117820190378618')*z**12 +\
R('6.9907892007964976766732412698483437200042427956877048')*z**13 -\
R('6.8505136291969230335492335563142032652696292334915144')*z**14 +\
R('5.6204644779027399939285962317939158700310771726450817')*z**15 -\
R('3.7571225990700255428818131364610970583607680405651151')*z**16 +\
R('1.9818690236901859077542482819362263892272661043080687')*z**17 -\
R('0.79042000831079783891503600943029279816999622133364452')*z**18 +\
R('0.22329682322488738828626081685481860397821781004733237')*z**19 -\
R('0.039725533876212767966626827576173773810516148708390759')* z**20 +\
R('0.0033403676833118318893558876073892257038276364922004966')*z**21
print("s")

coeffs = expr3.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        # print_dyadic(c)
        # print(c)
        print_double_double("", c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")

expr4 = R(1) + R('8.5611524981960027614349434134813648730968871378154996') * z +\
R('16.891526308836850603855451533848752349480326414401464') * z**2

print("s")

coeffs = expr4.list()  # This gives all coefficients up to the current precision
for i, c in enumerate(coeffs):
    if c != 0:
        # print_dyadic(c)
        print_double_double("", c)
        # print(c)
        # print("f64::from_bits(" + double_to_hex(c) + "),")

(0x3c1be095d05c0a81, 0x3fbdadb014541eb2),
(0x3c6037c12ba0b815, 0x3fd1dadb014541eb),
(0x3c2037c12ba0b815, 0x3f99dadb014541eb),
(0xbbe07eec845045e4, 0x3f4bb90e85debf56),
(0x3b830f4c5f3df300, 0x3eef4747696cf839),
(0x3b252fcaeee73fd1, 0x3e85d6b13b0d88ca),
(0xbab3127e7a5114b0, 0x3e14c2b6e8177e1a),
(0x3a309e5ad5685b51, 0x3d9ca0246d234e72),
(0xb9a183e5b5dac36d, 0x3d1df24eb119a2f9),
(0xb93ad95e64dfe5fc, 0x3c9896d55d330a18),
s
(0xbc97b64b25244ad6, 0x3ff40d931ff62706),
(0x3c9a1688cf334c34, 0x4026664d0189916c),
(0xbcc7e17378873c58, 0x40390c1509d1e208),
(0x3cbc4973f3c5d670, 0x401b3d731b85f2ac),
(0x3c933564234461b2, 0xbff8fb80e04d69cf),
(0x3c82498fbf50fe90, 0x3fef167068283d7e),
(0x3c2f4dc704d7c59d, 0xbfed83372b6a7c7e),
(0x3c68ea0b746e2ab0, 0x3ff1d3dae3d65ebf),
(0xbc84fb7c307a8dcd, 0xbff8b994d39a4ae3),
(0x3ca4e1e7f9af4249, 0x4002412f816c0a67),
(0x3caa7719cafdd0de, 0xc00b00cc9475d532),
(0x3cb818338b2aa469, 0x4013076e044ea75a),
(0xbc92861cf8b4f4df, 0xc0188742789f0db3),
(0xbcb865210d89e7f7, 0x401bf6917